# Main notebook for COVID-19 detection in children

## Authors:

- Marc Garcia
- Jofre Poch
- Pau Tarragó
- Pau Matas
- Tomás Gadea

## Summary of steps for the project:
- **Exploratory Data Analysis (EDA)**: entendre les dades


- **Data Cleaning**: treure NULLs, eliminar dades no importants, etc.


- **Feature Selection**: escollir les variables explicatives que li passarem al model


- **Model selection**: Escollir un model classificador (té covid o no; si té covid => PCR, si no => pa casa)


- **Model training**: entrenar el model, és copiar 4 linies de codi xd


- **Model Validation**: Veure com de bé ho ha fet. Per exemple veient els square errors de valors predits vs resposta de veritat.


- **Parameter tunning**: Canviar els paràmetres per millorar el model validation.

## Actual work: